In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 读取数据C:\MyProject\MIMIC-IPE\data\base10.csv
data = pd.read_csv('C:\\MyProject\\MIMIC-IPE\\data\\base10.csv')
# 删除data中所有列名最后两位字符串为“_1”的列
data = data.drop(data.columns[data.columns.str.endswith('_1')], axis=1)
# 删除data中所有列名最后两位字符串为“_2”的列
data = data.drop(data.columns[data.columns.str.endswith('_2')], axis=1)
# 设置subject_id、hadm_id和stay_id为索引
data = data.set_index(['subject_id'])

In [23]:
'''
删除以下列：
hadm_id
stay_id
blood_count_hematocrit
blood_count_hemoglobin
blood_count_mch
blood_count_mchc
blood_count_mcv
blood_count_platelet
blood_count_rbc
blood_count_rdw
blood_count_wbc
coagulation_ptt
'''
data = data.drop(['hadm_id', 'stay_id', 'blood_count_hematocrit', 'blood_count_hemoglobin', 'blood_count_mch',
                  'blood_count_mchc', 'blood_count_mcv', 'blood_count_platelet', 'blood_count_rbc',
                  'blood_count_rdw', 'blood_count_wbc', 'coagulation_ptt'], axis=1)

In [24]:
'''删除以下列：
hospstay_seq
first_hosp_stay
icustay_seq
first_icu_stay
icd_code
icd_version

dischtime
icu_intime
avg_los
'''
data = data.drop(['hospstay_seq', 'first_hosp_stay', 'icustay_seq', 'first_icu_stay', 'icd_code', 'icd_version',
                  'dischtime', 'icu_intime', 'avg_los'], axis=1)
'''提取以下几列，以下几列是待比较的值，形成一个新的dataframe：
hospital_expire_flag
re_inhosp
icu_count
14-days
31-days
los_icu
'''
data_compare = data[['hospital_expire_flag', 're_inhosp', 'icu_count', '14-days', '31-days', 'los_icu']]
# 从data中删除以上列
data = data.drop(['hospital_expire_flag', 're_inhosp', 'icu_count', '14-days', '31-days', 'los_icu'], axis=1)

In [25]:
# data中的gender列使用labelencoder进行编码
from sklearn.preprocessing import LabelEncoder

# Instantiate the LabelEncoder
labelencoder = LabelEncoder()
# Fit and transform the 'gender' column in 'X'
data['gender'] = labelencoder.fit_transform(data['gender'])
# data中的Group_label列使用labelencoder进行编码
# Fit and transform the 'Group_label' column in 'X'
data['Group_Label'] = labelencoder.fit_transform(data['Group_Label'])
# Decode the encoded 'Group_Label' column and print the results
decoded_labels = labelencoder.inverse_transform(data['Group_Label'])
encoded_labels = data['Group_Label'].unique()
# 使用 inverse_transform 方法将数字编码还原为原始标签
original_labels = labelencoder.inverse_transform(encoded_labels)
# 打印出数字编码及其对应的原始标签
for encoded, original in zip(encoded_labels, original_labels):
    print(f'{encoded}: {original}')
# X中的race列使用one-hot编码，然后再将原race列删除，再把one-hot编码后的列加入X
# Perform one-hot encoding on the 'race' column
race_dummies = pd.get_dummies(data['race'], prefix='race')
# Drop the original 'race' column from 'data'
data = data.drop('race', axis=1)
# Concatenate 'data' with the one-hot encoded DataFrame
data = pd.concat([data, race_dummies], axis=1)

0: DVT-PE
1: IPE


In [15]:
# 转换日期格式为日期时间格式
data['dod'] = pd.to_datetime(data['dod'])
data['admittime'] = pd.to_datetime(data['admittime'])

# 计算事件发生或观察结束的天数
data['obs_days'] = (data['dod'] - data['admittime']).dt.days

# 创建事件状态列，1表示在观察期间内发生了感兴趣的事件，0表示没有或数据被删失
data['event_status'] = np.where(data['dod'].notnull(), 1, 0)

# 删除不再需要的列
data = data.drop('admittime', axis=1)



In [16]:
_# 获取data中的所有列名、列中空值计数和列中空值占比，形成一个新的DataFrame
data_info = pd.DataFrame(data.columns, columns=['column_name'])
data_info['missing_count'] = data.isnull().sum().values
# 将空值占比转换为百分数形式
data_info['missing_rate'] = (data.isnull().sum().values / data.shape[0] * 100).round(2)

In [17]:
# 将data保存到C:\MyProject\MIMIC-IPE\data\full_dataforcox.csv
data.to_csv('C:/MyProject/MIMIC-IPE/data/full_dataforcox.csv')

In [22]:
# data中event_status列为事件列，death_day列为时间列，其余列为特征列，Group_Label为分组列，对data进行Kaplan-Meier生存曲线分析，使用log-rank检验，并且画出生存曲线

from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

# 初始化KaplanMeierFitter类
kmf = KaplanMeierFitter()

# 对每个组进行Kaplan-Meier生存曲线分析，并画出生存曲线
groups = data['Group_Label'].unique()
for i, group in enumerate(groups):
    group_data = data[data['Group_Label'] == group]
    kmf.fit(group_data['death_time'], group_data['7-days'], label=str(group))
    kmf.plot()

# 对每两个组进行log-rank检验
for i in range(len(groups)):
    for j in range(i+1, len(groups)):
        group_i_data = data[data['Group_Label'] == groups[i]]
        group_j_data = data[data['Group_Label'] == groups[j]]
        results = logrank_test(group_i_data['death_time'], group_j_data['death_time'],
                               event_observed_A=group_i_data['7-days'], event_observed_B=group_j_data['7-days'])
        print(f'Log-rank test between group {groups[i]} and group {groups[j]}: p={results.p_value}')

TypeError: NaNs were detected in the dataset. Try using pd.isnull to find the problematic values.